In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [11]:
파일경로 = '/content/drive/MyDrive/time.xlsx'
time = pd.read_excel(파일경로)
time_in = pd.get_dummies(time['세탁기사용량'])
print(time.head())
print(time_in.head())

                 결제시간  세탁기사용량
0 2025-04-01 00:00:00       1
1 2025-04-01 00:05:00       2
2 2025-04-01 00:10:00       2
3 2025-04-01 00:15:00       1
4 2025-04-01 00:20:00       2
      0      1      2      3      4      5      6      7      8      9   \
0  False   True  False  False  False  False  False  False  False  False   
1  False  False   True  False  False  False  False  False  False  False   
2  False  False   True  False  False  False  False  False  False  False   
3  False   True  False  False  False  False  False  False  False  False   
4  False  False   True  False  False  False  False  False  False  False   

      10     11     12     13  
0  False  False  False  False  
1  False  False  False  False  
2  False  False  False  False  
3  False  False  False  False  
4  False  False  False  False  


In [12]:
# 1. 시간 (Hour): 0 ~ 23시 (주기: 24)
time['hour_sin'] = np.sin(2 * np.pi * time['결제시간'].dt.hour / 24)
time['hour_cos'] = np.cos(2 * np.pi * time['결제시간'].dt.hour / 24)

# 2. 분 (Minute): 0 ~ 59분 (주기: 60)
time['minute_sin'] = np.sin(2 * np.pi * time['결제시간'].dt.minute / 60)
time['minute_cos'] = np.cos(2 * np.pi * time['결제시간'].dt.minute / 60)

# 3. 요일 (Day of Week): 월(0) ~ 일(6) (주기: 7)
time['dayofweek_sin'] = np.sin(2 * np.pi * time['결제시간'].dt.dayofweek / 7)
time['dayofweek_cos'] = np.cos(2 * np.pi * time['결제시간'].dt.dayofweek / 7)

# 4. 월 (Month): 1 ~ 12월 (주기: 12)
time['month_sin'] = np.sin(2 * np.pi * time['결제시간'].dt.month / 12)
time['month_cos'] = np.cos(2 * np.pi * time['결제시간'].dt.month / 12)

print(time.head())

                 결제시간  세탁기사용량  hour_sin  hour_cos  minute_sin    minute_cos  \
0 2025-04-01 00:00:00       1       0.0       1.0    0.000000  1.000000e+00   
1 2025-04-01 00:05:00       2       0.0       1.0    0.500000  8.660254e-01   
2 2025-04-01 00:10:00       2       0.0       1.0    0.866025  5.000000e-01   
3 2025-04-01 00:15:00       1       0.0       1.0    1.000000  2.832769e-16   
4 2025-04-01 00:20:00       2       0.0       1.0    0.866025 -5.000000e-01   

   dayofweek_sin  dayofweek_cos  month_sin  month_cos  
0       0.781831        0.62349   0.866025       -0.5  
1       0.781831        0.62349   0.866025       -0.5  
2       0.781831        0.62349   0.866025       -0.5  
3       0.781831        0.62349   0.866025       -0.5  
4       0.781831        0.62349   0.866025       -0.5  


In [13]:
X = time.drop(columns=['세탁기사용량', '결제시간'])
y = time_in
print(X.head())
print(y.head())

   hour_sin  hour_cos  minute_sin    minute_cos  dayofweek_sin  dayofweek_cos  \
0       0.0       1.0    0.000000  1.000000e+00       0.781831        0.62349   
1       0.0       1.0    0.500000  8.660254e-01       0.781831        0.62349   
2       0.0       1.0    0.866025  5.000000e-01       0.781831        0.62349   
3       0.0       1.0    1.000000  2.832769e-16       0.781831        0.62349   
4       0.0       1.0    0.866025 -5.000000e-01       0.781831        0.62349   

   month_sin  month_cos  
0   0.866025       -0.5  
1   0.866025       -0.5  
2   0.866025       -0.5  
3   0.866025       -0.5  
4   0.866025       -0.5  
      0      1      2      3      4      5      6      7      8      9   \
0  False   True  False  False  False  False  False  False  False  False   
1  False  False   True  False  False  False  False  False  False  False   
2  False  False   True  False  False  False  False  False  False  False   
3  False   True  False  False  False  False  False  False

In [16]:
input = tf.keras.layers.Input(shape = [8])
x = tf.keras.layers.LeakyReLU(alpha = 0.01)(input)
hidden = tf.keras.layers.Dense(64, kernel_initializer='he_normal')(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.01)(hidden)
hidden = tf.keras.layers.Dense(64, kernel_initializer='he_normal')(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.01)(hidden)
hidden = tf.keras.layers.Dense(64, kernel_initializer='he_normal')(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.01)(hidden)
hidden = tf.keras.layers.Dense(64, kernel_initializer='he_normal')(x)
output = tf.keras.layers.Dense(14, activation = 'softmax')(hidden)

model = tf.keras.models.Model(input, output)

model.compile(optimizer = 'adagrad', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(X,y,epochs = 1000, verbose = 0)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [20]:
result = model.predict(X[-5:])
print(np.argmax(result, axis=1))
print(y[-5:])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[1 1 1 1 1]
         0      1      2      3      4      5      6      7      8      9   \
8647  False  False  False  False  False   True  False  False  False  False   
8648  False  False  False   True  False  False  False  False  False  False   
8649  False   True  False  False  False  False  False  False  False  False   
8650  False   True  False  False  False  False  False  False  False  False   
8651   True  False  False  False  False  False  False  False  False  False   

         10     11     12     13  
8647  False  False  False  False  
8648  False  False  False  False  
8649  False  False  False  False  
8650  False  False  False  False  
8651  False  False  False  False  
